In [3]:
from datetime import datetime, timedelta
import pandas as pd
from tqdm import tqdm
from modeler.modeler import Modeler as m
from processor.processor import Processor as p
from database.market import Market
from database.adatabase import ADatabase
import numpy as np

In [4]:
market = Market()
speculation_db = ADatabase("speculation")
financial_db = ADatabase("financial")

In [5]:
market.connect()
sp500 = market.retrieve("sp500")
prices = market.retrieve("prices")
market.disconnect()

In [6]:
prices = p.column_date_processing(prices)
prices["year"] = [x.year for x in prices["date"]]
prices["week"] = [x.week for x in prices["date"]]

In [8]:
prices["quarter"] = [x.quarter for x in prices["date"]]

In [9]:
speculation_db.connect()
spec_sim = speculation_db.retrieve("sim")
speculation_db.disconnect()
financial_db.connect()
fin_sim = speculation_db.retrieve("sim")
financial_db.disconnect()

In [11]:
spec_sim["week"] =  spec_sim["week"] + 1

In [17]:
training_year = 4
spec_sim["weekly_prediction"] = (spec_sim["cat_prediction"] + spec_sim["skl_prediction"] + spec_sim["xgb_prediction"]) / 3
fin_sim["quarterly_prediction"] = (fin_sim["cat_prediction"] + fin_sim["skl_prediction"] + fin_sim["xgb_prediction"]) / 3

In [18]:
sim = prices.merge(spec_sim[spec_sim["training_year"]==training_year],on=["year","week","ticker"],how="left") \
                    .merge(fin_sim[fin_sim["training_year"]==training_year],on=["year","week","ticker"],how="left").dropna()

sim["weekly_delta"] = (sim["weekly_prediction"] - sim["adjclose"]) / sim["adjclose"]
sim["quarterly_delta"] = (sim["quarterly_prediction"] - sim["adjclose"]) / sim["adjclose"]

In [35]:
sim["timing"] = sim["weekly_prediction"] < sim["quarterly_prediction"]

In [44]:
sim[(sim["timing"]==True) & (sim["quarterly_delta"]>=0.5)].sort_values("date")[["date","ticker","adjclose","weekly_prediction","quarterly_prediction"]]

,date,ticker,adjclose,weekly_prediction,quarterly_prediction
145598,2016-02-22,AMD,2.010000,2.606694,3.271103
145599,2016-02-23,AMD,1.970000,2.606694,3.271103
145600,2016-02-24,AMD,2.020000,2.606694,3.271103
145601,2016-02-25,AMD,2.030000,2.606694,3.271103
145602,2016-02-26,AMD,2.070000,2.606694,3.271103
...,...,...,...,...,...
1462474,2020-11-11,MRO,4.939549,6.362963,7.463067
1462475,2020-11-12,MRO,4.695981,6.362963,7.463067
1462476,2020-11-13,MRO,4.929806,6.362963,7.463067
282274,2022-05-23,AZO,1805.220000,2408.310723,2880.741730


In [26]:
positions = 5
start_date = datetime(2016,1,1)
end_date = datetime(2022,6,30)
speculation_db.connect()
value = True
for signal in tqdm([0.5,1]):
    for req in tqdm([0.5]):
        for position in range(positions):
            date = start_date
            iterration_sim = sim.copy()
            while date < end_date:
                try:
                    todays_recs = iterration_sim[(iterration_sim["date"]==date)]
                    todays_recs.sort_values("weekly_delta",ascending=False,inplace=True)
                    if todays_recs.index.size > 0 and date.weekday() <= 2:
                        offering = todays_recs.iloc[position]
                        if (offering["quarterly_delta"] >= signal) and (offering["weekly_prediction"] <= 0):
                            trade = offering
                            ticker = trade["ticker"]
                            buy_price = trade["adjclose"]
                            quarter = date.month // 3 + 1
                            ldq = datetime(date.year,(quarter*3),30)
                            end_index = int((ldq - date).days)
                            exits = iterration_sim[(iterration_sim["ticker"]==ticker) & (iterration_sim["date"]>date)].iloc[:end_index]
                            exits["gains"] = (exits["adjclose"] - buy_price) / buy_price
                            gain_exits = exits[exits["gains"]>=req].sort_values("date")
                            if gain_exits.index.size < 1:
                                exit = iterration_sim[(iterration_sim["ticker"]==ticker) & \
                                                      (iterration_sim["date"]>date)].iloc[end_index:].iloc[0]
                                trade["sell_price"] = exit["adjclose"]
                            else:
                                exit = gain_exits.iloc[0]
                                trade["sell_price"] = buy_price * (1+(req))
                            delta = (trade["sell_price"] - buy_price) / buy_price
                            trade["sell_date"] = exit["date"]
                            date = exit["date"] + timedelta(days=1)
                            trade["projected_delta"] = offering["weekly_delta"]
                            trade["delta"] = delta
                            trade["position"] = position
                            trade["req"] = req
                            trade["value"] = value
                            trade["training_year"] = training_year
                            trade["signal"] = signal
                            included = ["date","sell_date","ticker","adjclose","sell_price","delta"
                                        ,"projected_delta","position"
                                        ,"req","signal","value","training_year"]
                            trade_record = pd.DataFrame([trade]).reset_index(drop=True)[included]
                            speculation_db.store("timing_trades",trade_record)
                        else:
                            date = date + timedelta(days=1)
                    else:
                        date = date + timedelta(days=1)
                except Exception as e:
                    print(str(e))
                    date = date+timedelta(days=1)
speculation_db.disconnect()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:50<00:00, 25.20s/it]
